In [50]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
import matplotlib.pyplot as plt
import colorcet as cc
import networkx as nx
import pandas as pd
import plotly.graph_objects as go
import sys
from dash import Dash, html
import dash_cytoscape as cytov
import igraph as ig
from IPython.display import Image
import colorcet as cc

sys.path.append("/Users/jlq293/Projects/Study-1-Bibliometrics/src/main_path/")

from MainPathReader import MainPathReader
from MainPathDataAssigner import MainPathDataAssigner
from MainPathPlotterStatic import MainPathPlotterStatic
from MainPathPlotterInteractive import MainPathPlotterInteractive

# load and prep data


In [44]:
params = "alpha0.3_k20_res0.005"
p = f"../data/06-clustered-df/{params}.pkl"
df = pd.read_pickle(p)

path_to_main_path = "../data/08-main-paths/final_mp_clustered.graphml"
G = nx.read_graphml(path_to_main_path)

list(G.nodes(data=True))[0]

('1',
 {'label': 'Magnussen_1982',
  'eid': '2-s2.0-0019996341',
  'title': 'Treatment of myoclonic syndromes with paroxetine alone or combined with 5‐HTP',
  'citedby_count': '13',
  'doi': '10.1111/j.1600-0404.1982.tb04525.x',
  'year': '1982',
  'cluster_alpha0.3_k20_res0.005': '36',
  'centrality_alpha0.3_k20_res0.005': '0.05427762074604986',
  'full_label': '36; Serotonin Binding; Animal Studies,'})

In [45]:
def node_and_edge_list_fun(G):
    nodes_list = []
    for node, data in G.nodes(data=True):
        dat = {
            "id": node,
            "cluster": data["cluster_alpha0.3_k20_res0.005"],
            "cluster_label": data["full_label"],
            "title": data["title"],
        }
        pos = {"x": 0, "y": 0}
        nodes_list.append({"data": dat, "position": pos})

    edges_list = []

    for source, target in G.edges:
        edges_list.append({"data": {"source": target, "target": source}})
    print("Node example")
    print(nodes_list[0])
    return nodes_list, edges_list


def default_stylesheet_fun(cluster_color_dict):
    default_stylesheet = [
        {
            "selector": "node",  # Select all nodes
            "style": {
                "shape": "rectangle",  # Change the shape to rectangle
                "width": "80px",  # Specify width of the nodes
                "height": "30px",  # Specify height of the nodes
                "label": "data(cluster)",  # Show node label
                "font-size": "24px",  # Set font size
                "text-valign": "center",  # Vertically center the label
                "text-valign": "center",
                "text-halign": "center",
            },
        },
        {
            "selector": "edge",
            "style": {
                # The default curve style does not work with certain arrows
                "curve-style": "bezier",
                "source-arrow-shape": "triangle",
                # "target-arrow-shape": "triangle",
            },
        },
    ]

    # Add styles for each label-color mapping
    for cluster, color in cluster_color_dict.items():
        default_stylesheet.append(
            {
                "selector": f'[cluster = "{cluster}"]',  # Select nodes with the specific label
                "style": {"background-color": color, "line-color": color},
            }
        )
    return default_stylesheet


def cluster_color_dict(G):
    cluster_color_dict = {
        cluster: cc.glasbey_light[i]
        for i, cluster in enumerate(
            set(nx.get_node_attributes(G, "cluster_alpha0.3_k20_res0.005").values())
        )
    }
    # print first
    print(f"Cluster color dictionary example: {list(cluster_color_dict.items())[:5]}")

    return cluster_color_dict

In [46]:
nodes_list, edges_list = node_and_edge_list_fun(G)

cluster_color_dict = cluster_color_dict(G)

default_stylesheet = default_stylesheet_fun(cluster_color_dict)

Node example
{'data': {'id': '1', 'cluster': '36', 'cluster_label': '36; Serotonin Binding; Animal Studies,', 'title': 'Treatment of myoclonic syndromes with paroxetine alone or combined with 5‐HTP'}, 'position': {'x': 0, 'y': 0}}
Cluster color dictionary example: [('2', '#d60000'), ('5', '#018700'), ('116', '#b500ff'), ('29', '#05acc6'), ('19', '#97ff00')]


In [91]:
from dash import Dash, html
import dash_cytoscape as cyto

# nodes_list[0] = {'data': {'id': '1', 'label': '36', 'cluster': '36'}, 'position': {'x': 0, 'y': 0}}
cyto.load_extra_layouts()

app = Dash(__name__)
server = app.server


elements = nodes_list + edges_list

app.layout = html.Div(
    [
        html.Div(
            cyto.Cytoscape(
                id="main-path-cytoscape",
                elements=elements,
                style={
                    "width": "100%",
                    "height": "800px",
                    "background-color": "#f5f5f5",  # Change background color here
                    "padding": "20px",
                },
                layout={"name": "dagre"},
                stylesheet=default_stylesheet,
                #                responsive=True,
            ),
        ),
        html.Button("Print elements JSONified", id="button-main-path-cytoscape"),
        html.Div(id="html-main-path-cytoscape"),
    ]
)


@app.callback(
    Output("html-main-path-cytoscape", "children"),
    [Input("button-main-path-cytoscape", "n_clicks")],
    [State("main-path-cytoscape", "elements")],
)
def testCytoscape(n_clicks, elements):
    if n_clicks:
        return json.dumps(elements)


if __name__ == "__main__":
    app.run(debug=True)

# callbacks


In [48]:
from dash import Dash, html, dcc, Input, Output, callback
import dash_cytoscape as cyto

# nodes_list[0] = {'data': {'id': '1', 'label': '36', 'cluster': '36'}, 'position': {'x': 0, 'y': 0}}
cyto.load_extra_layouts()

app = Dash(__name__)
server = app.server


elements = nodes_list + edges_list

layout_options = [
    "dagre",
    "klay",
    # "cose-bilkent",
    # "cola",
    # "euler",
    # "spread",
    # "cose",
    "random",
    # "circle",
    # "cose",
    "concentric",
]

app.layout = html.Div(
    [
        dcc.Dropdown(
            id="dropdown-update-layout",
            value="dagre",
            clearable=False,
            options=[
                {"label": name.capitalize(), "value": name} for name in layout_options
            ],
        ),
        html.Div(
            cyto.Cytoscape(
                id="cytoscape-update-layout",
                elements=elements,
                style={
                    "width": "100%",
                    "height": "800px",
                    "background-color": "#f5f5f5",  # Change background color here
                    "padding": "20px",
                },
                layout={"name": "dagre"},
                stylesheet=default_stylesheet,
                #                responsive=True,
            ),
        ),
    ]
)


@callback(
    Output("cytoscape-update-layout", "layout"),
    Input("dropdown-update-layout", "value"),
)
def update_layout(layout):
    return {"name": layout, "animate": True}


# if __name__ == "__main__":
#    app.run(debug=True)

In [90]:
from dash import Dash, html, dcc, Input, Output, callback
import dash_cytoscape as cyto

# nodes_list[0] = {'data': {'id': '1', 'label': '36', 'cluster': '36'}, 'position': {'x': 0, 'y': 0}}
cyto.load_extra_layouts()

app = Dash(__name__)
server = app.server


elements = nodes_list + edges_list

layout_options = [
    "dagre",
    "klay",
    # "cose-bilkent",
    # "cola",
    # "euler",
    # "spread",
    # "cose",
    "random",
    # "circle",
    # "cose",
    "concentric",
]

app.layout = html.Div(
    [
        dcc.Dropdown(
            id="dropdown-update-layout",
            value="dagre",
            clearable=False,
            options=[
                {"label": name.capitalize(), "value": name} for name in layout_options
            ],
        ),
        html.Div(
            cyto.Cytoscape(
                id="main-path-cytoscape",
                elements=elements,
                style={
                    "width": "100%",
                    "height": "800px",
                    "background-color": "#f5f5f5",  # Change background color here
                    "padding": "20px",
                },
                layout={"name": "dagre"},
                stylesheet=default_stylesheet,
                #                responsive=True,
            ),
        ),
        html.Button("Print elements JSONified", id="button-main-path-cytoscape"),
        html.Div(id="html-main-path-cytoscape"),
    ]
)


@app.callback(
    Output("html-main-path-cytoscape", "children"),
    [Input("button-main-path-cytoscape", "n_clicks")],
    [State("main-path-cytoscape", "elements")],
)
def testCytoscape(n_clicks, elements):
    if n_clicks:
        return json.dumps(elements)


@callback(
    Output("main-path-cytoscape", "layout"),
    Input("dropdown-update-layout", "value"),
)
def update_layout(layout):
    return {"name": layout, "animate": True}


if __name__ == "__main__":
    app.run(debug=True)

In [92]:
elements_copied = [
    {
        "data": {
            "id": "1",
            "cluster": "36",
            "cluster_label": "36; Serotonin Binding; Animal Studies,",
            "title": "Treatment of myoclonic syndromes with paroxetine alone or combined with 5\u2010HTP",
        },
        "position": {"x": 672.5, "y": 15},
    },
    {
        "data": {
            "id": "2",
            "cluster": "36",
            "cluster_label": "36; Serotonin Binding; Animal Studies,",
            "title": "Paroxetine, a potent selective long-acting inhibitor of synaptosomal 5-HT uptake in mice",
        },
        "position": {"x": 672.5, "y": 95},
    },
    {
        "data": {
            "id": "3",
            "cluster": "54",
            "cluster_label": "54; Fluvoxamine Depression Treatment",
            "title": "Fluvoxamine and chlorimipramine in endogenous depression",
        },
        "position": {"x": 467.5, "y": 15},
    },
    {
        "data": {
            "id": "4",
            "cluster": "36",
            "cluster_label": "36; Serotonin Binding; Animal Studies,",
            "title": "High affinity binding of [<sup>3</sup>H]paroxetine and [<sup>3</sup>H]imipramine to human platelet membranes",
        },
        "position": {"x": 672.5, "y": 175},
    },
    {
        "data": {
            "id": "5",
            "cluster": "54",
            "cluster_label": "54; Fluvoxamine Depression Treatment",
            "title": "A double\u2010blind controlled clinical trial comparing fluvoxamine with imipramine.",
        },
        "position": {"x": 412.5, "y": 95},
    },
    {
        "data": {
            "id": "6",
            "cluster": "5",
            "cluster_label": "5; Pharmacokinetics; Cytochrome P450 System; Drug Metabolism",
            "title": "Fluvoxamine maleate; Disposition in man",
        },
        "position": {"x": 542.5, "y": 95},
    },
    {
        "data": {
            "id": "7",
            "cluster": "5",
            "cluster_label": "5; Pharmacokinetics; Cytochrome P450 System; Drug Metabolism",
            "title": "Fluvoxamine maleate: metabolism in man",
        },
        "position": {"x": 542.5, "y": 175},
    },
    {
        "data": {
            "id": "8",
            "cluster": "54",
            "cluster_label": "54; Fluvoxamine Depression Treatment",
            "title": "Clinical trials of fluvoxamine vs chlorimipramine with single and three times daily dosing.",
        },
        "position": {"x": 402.5, "y": 175},
    },
    {
        "data": {
            "id": "9",
            "cluster": "18",
            "cluster_label": "18; Cardiovascular Risk",
            "title": "Cardiac effects of antidepressant drugs. A comparison of the tricyclic antidepressants and fluvoxamine.",
        },
        "position": {"x": 402.5, "y": 255},
    },
    {
        "data": {
            "id": "10",
            "cluster": "36",
            "cluster_label": "36; Serotonin Binding; Animal Studies,",
            "title": "Imipramine binding site Temperature dependence of the binding of <sup>3</sup>H-labeled imipramine and <sup>3</sup>H-labeled paroxetine to human platelet membrane",
        },
        "position": {"x": 672.5, "y": 255},
    },
    {
        "data": {
            "id": "11",
            "cluster": "36",
            "cluster_label": "36; Serotonin Binding; Animal Studies,",
            "title": "Size determination of binding polymers for [<sup>3</sup>H]imipramine and [<sup>3</sup>H]paroxetine in human platelet membranes",
        },
        "position": {"x": 672.5, "y": 335},
    },
    {
        "data": {
            "id": "12",
            "cluster": "54",
            "cluster_label": "54; Fluvoxamine Depression Treatment",
            "title": "The tolerance and antidepressive activity of fluvoxamine as a single dose compared to a twice daily dose",
        },
        "position": {"x": 542.5, "y": 255},
    },
    {
        "data": {
            "id": "13",
            "cluster": "36",
            "cluster_label": "36; Serotonin Binding; Animal Studies,",
            "title": "Characterization of [<sup>3</sup>H]paroxetine binding to rat cortical membranes",
        },
        "position": {"x": 672.5, "y": 415},
    },
    {
        "data": {
            "id": "14",
            "cluster": "54",
            "cluster_label": "54; Fluvoxamine Depression Treatment",
            "title": "Fluvoxamine: A Review of Its Pharmacodynamic and Pharmacokinetic Properties, and Therapeutic Efficacy in Depressive Illness",
        },
        "position": {"x": 467.5, "y": 335},
    },
    {
        "data": {
            "id": "15",
            "cluster": "36",
            "cluster_label": "36; Serotonin Binding; Animal Studies,",
            "title": "Temperature Dependence of Drug Interaction with the Platelet 5\u2010Hydroxytryptamine Transporter: A Clue to the Imipramine Selectivity Paradox",
        },
        "position": {"x": 672.5, "y": 495},
    },
    {
        "data": {
            "id": "16",
            "cluster": "36",
            "cluster_label": "36; Serotonin Binding; Animal Studies,",
            "title": "Regional and subcellular localization in human brain of [<sup>3</sup>H]paroxetine binding, a marker of serotonin uptake sites",
        },
        "position": {"x": 672.5, "y": 575},
    },
    {
        "data": {
            "id": "17",
            "cluster": "19",
            "cluster_label": "19; Paroxetine Depression Treatment",
            "title": "An overview of the animal pharmacology of paroxetine",
        },
        "position": {"x": 672.5, "y": 655},
    },
    {
        "data": {
            "id": "18",
            "cluster": "54",
            "cluster_label": "54; Fluvoxamine Depression Treatment",
            "title": "Paroxetine: A selective serotonin reuptake inhibitor showing better tolerance, but weaker antidepressant effect than clomipramine in a controlled multicenter study",
        },
        "position": {"x": 467.5, "y": 415},
    },
    {
        "data": {
            "id": "19",
            "cluster": "21",
            "cluster_label": "21; Pain Management",
            "title": "The selective serotonin reuptake inhibitor paroxetine is effective in the treatment of diabetic neuropathy symptoms",
        },
        "position": {"x": 467.5, "y": 495},
    },
    {
        "data": {
            "id": "20",
            "cluster": "5",
            "cluster_label": "5; Pharmacokinetics; Cytochrome P450 System; Drug Metabolism",
            "title": "Extremely slow metabolism of amitriptyline but normal metabolism of imipramine and desipramine in an extensive metabolizer of sparteine, debrisoquine, and mephenytoin",
        },
        "position": {"x": 467.5, "y": 575},
    },
    {
        "data": {
            "id": "21",
            "cluster": "5",
            "cluster_label": "5; Pharmacokinetics; Cytochrome P450 System; Drug Metabolism",
            "title": "Fluoxetine and norfluoxetine are potent inhibitors of P450IID6\u2010the source of the sparteine/debrisoquine oxidation polymorphism [letter]",
        },
        "position": {"x": 542.5, "y": 655},
    },
    {
        "data": {
            "id": "22",
            "cluster": "5",
            "cluster_label": "5; Pharmacokinetics; Cytochrome P450 System; Drug Metabolism",
            "title": "Pharmacokinetics of the selective serotonin reuptake inhibitor paroxetine: Nonlinearity and relation to the sparteine oxidation polymorphism;The relationship between paroxetine and the sparteine oxidation polymorphism",
        },
        "position": {"x": 532.5, "y": 735},
    },
    {
        "data": {
            "id": "23",
            "cluster": "5",
            "cluster_label": "5; Pharmacokinetics; Cytochrome P450 System; Drug Metabolism",
            "title": "The role of cytochrome P4502D6 in the metabolism of paroxetine by human liver microsomes.",
        },
        "position": {"x": 532.5, "y": 815},
    },
    {
        "data": {
            "id": "24",
            "cluster": "5",
            "cluster_label": "5; Pharmacokinetics; Cytochrome P450 System; Drug Metabolism",
            "title": "The effect of selective serotonin re\u2010uptake inhibitors on cytochrome P4502D6 (CYP2D6) activity in human liver microsomes.",
        },
        "position": {"x": 532.5, "y": 895},
    },
    {
        "data": {
            "id": "25",
            "cluster": "5",
            "cluster_label": "5; Pharmacokinetics; Cytochrome P450 System; Drug Metabolism",
            "title": "Inhibitors of imipramine metabolism by human liver microsomes.",
        },
        "position": {"x": 607.5, "y": 975},
    },
    {
        "data": {
            "id": "26",
            "cluster": "5",
            "cluster_label": "5; Pharmacokinetics; Cytochrome P450 System; Drug Metabolism",
            "title": "Fluvoxamine is a potent inhibitor of cytochrome P4501A2",
        },
        "position": {"x": 672.5, "y": 1055},
    },
    {
        "data": {
            "id": "27",
            "cluster": "5",
            "cluster_label": "5; Pharmacokinetics; Cytochrome P450 System; Drug Metabolism",
            "title": "An interaction between carbamazepine and fluvoxamine",
        },
        "position": {"x": 672.5, "y": 1135},
    },
    {
        "data": {
            "id": "28",
            "cluster": "Family",
            "cluster_label": "5;64; Pharmacokinetics; Cytochrome P450 System; Drug Metabolism ::: Clinical Pharmacogenetics; Depression Treatment",
            "title": "Citalopram: Interaction studies with levomepromazine, imipramine, and lithium;Pharmacokinetics of citalopram in relation to the sparteine and the mephenytoin oxidation polymorphisms",
        },
        "position": {"x": 542.5, "y": 1215},
    },
    {
        "data": {
            "id": "29",
            "cluster": "5",
            "cluster_label": "5; Pharmacokinetics; Cytochrome P450 System; Drug Metabolism",
            "title": "Inhibition by paroxetine of desipramine metabolism in extensive but not in poor metabolizers of sparteine",
        },
        "position": {"x": 542.5, "y": 1295},
    },
    {
        "data": {
            "id": "30",
            "cluster": "5",
            "cluster_label": "5; Pharmacokinetics; Cytochrome P450 System; Drug Metabolism",
            "title": "Clinical Pharmacokinetics of Selective Serotonin Reuptake Inhibitors",
        },
        "position": {"x": 412.5, "y": 1135},
    },
    {
        "data": {
            "id": "31",
            "cluster": "54",
            "cluster_label": "54; Fluvoxamine Depression Treatment",
            "title": "Fluvoxamine: An Updated Review of its Pharmacology, and Therapeutic Use in Depressive Illness",
        },
        "position": {"x": 412.5, "y": 1215},
    },
    {
        "data": {
            "id": "32",
            "cluster": "54",
            "cluster_label": "54; Fluvoxamine Depression Treatment",
            "title": "Fluvoxamine: An Overview of its Pharmacological Properties and Review of its Therapeutic Potential in Non-Depressive Disorders",
        },
        "position": {"x": 412.5, "y": 1295},
    },
    {
        "data": {
            "id": "33",
            "cluster": "5",
            "cluster_label": "5; Pharmacokinetics; Cytochrome P450 System; Drug Metabolism",
            "title": "Pharmacokinetics of desipramine coadministered with sertraline or fluoxetine",
        },
        "position": {"x": 672.5, "y": 1375},
    },
    {
        "data": {
            "id": "34",
            "cluster": "5",
            "cluster_label": "5; Pharmacokinetics; Cytochrome P450 System; Drug Metabolism",
            "title": "Inhibition of diazepam metabolism by fluvoxamine: A pharmacokinetic study in normal volunteers",
        },
        "position": {"x": 282.5, "y": 1375},
    },
    {
        "data": {
            "id": "35",
            "cluster": "5",
            "cluster_label": "5; Pharmacokinetics; Cytochrome P450 System; Drug Metabolism",
            "title": "Inhibitors of alprazolam metabolism in vitro: effect of serotonin\u2010 reuptake\u2010inhibitor antidepressants, ketoconazole and quinidine.",
        },
        "position": {"x": 282.5, "y": 1215},
    },
    {
        "data": {
            "id": "36",
            "cluster": "61",
            "cluster_label": "61; Schizophrenia; Adjunctive Treatment",
            "title": "Fluvoxamine inhibition and carbamazepine induction of the metabolism of clozapine: Evidence from a therapeutic drug monitoring service",
        },
        "position": {"x": 672.5, "y": 1215},
    },
    {
        "data": {
            "id": "37",
            "cluster": "5",
            "cluster_label": "5; Pharmacokinetics; Cytochrome P450 System; Drug Metabolism",
            "title": "Selective serotonin reuptake inhibitors and theophylline metabolism in human liver microsomes: potent inhibition by fluvoxamine.",
        },
        "position": {"x": 672.5, "y": 1295},
    },
    {
        "data": {
            "id": "38",
            "cluster": "5",
            "cluster_label": "5; Pharmacokinetics; Cytochrome P450 System; Drug Metabolism",
            "title": "Metabolism of Drugs by Cytochrome P450 3A Isoforms: Implications for Drug Interactions in Psychopharmacology",
        },
        "position": {"x": 282.5, "y": 1295},
    },
    {
        "data": {
            "id": "39",
            "cluster": "5",
            "cluster_label": "5; Pharmacokinetics; Cytochrome P450 System; Drug Metabolism",
            "title": "Triazolam biotransformation by human liver microsomes in vitro: Effects of metabolic inhibitors and clinical confirmation of a predicted interaction with ketoconazole",
        },
        "position": {"x": 477.5, "y": 1375},
    },
    {
        "data": {
            "id": "40",
            "cluster": "5",
            "cluster_label": "5; Pharmacokinetics; Cytochrome P450 System; Drug Metabolism",
            "title": "Inhibition of Terfenadine Metabolism in Vitro by Azole Antifungal Agents and by Selective Serotonin Reuptake Inhibitor Antidepressants: Relation to Pharmacokinetic Interactions in Vivo",
        },
        "position": {"x": 542.5, "y": 1455},
    },
    {
        "data": {
            "id": "41",
            "cluster": "5",
            "cluster_label": "5; Pharmacokinetics; Cytochrome P450 System; Drug Metabolism",
            "title": "Phenacetin O-deethylation by human liver microsomes in vitro: Inhibition by chemical probes, SSRI antidepressants, nefazodone and venlafaxine",
        },
        "position": {"x": 542.5, "y": 1535},
    },
    {
        "data": {
            "id": "42",
            "cluster": "5",
            "cluster_label": "5; Pharmacokinetics; Cytochrome P450 System; Drug Metabolism",
            "title": "Human cytochromes mediating N-demethylation of fluoxetine in vitro",
        },
        "position": {"x": 542.5, "y": 1615},
    },
    {
        "data": {
            "id": "43",
            "cluster": "5",
            "cluster_label": "5; Pharmacokinetics; Cytochrome P450 System; Drug Metabolism",
            "title": "Pharmacokinetic drug interactions of new antidepressants: A review of the effects on the metabolism of other drugs",
        },
        "position": {"x": 412.5, "y": 1615},
    },
    {
        "data": {
            "id": "44",
            "cluster": "5",
            "cluster_label": "5; Pharmacokinetics; Cytochrome P450 System; Drug Metabolism",
            "title": "Drug interactions with newer antidepressants: Role of human cytochromes P450",
        },
        "position": {"x": 542.5, "y": 1695},
    },
    {
        "data": {
            "id": "45",
            "cluster": "5",
            "cluster_label": "5; Pharmacokinetics; Cytochrome P450 System; Drug Metabolism",
            "title": "Citalopram and desmethylcitalopram in vitro: Human cytochromes mediating transformation, and cytochrome inhibitory effects",
        },
        "position": {"x": 542.5, "y": 1775},
    },
    {
        "data": {
            "id": "46",
            "cluster": "5",
            "cluster_label": "5; Pharmacokinetics; Cytochrome P450 System; Drug Metabolism",
            "title": "Human cytochromes and some newer antidepressants: Kinetics, metabolism, and drug interactions",
        },
        "position": {"x": 542.5, "y": 1855},
    },
    {
        "data": {
            "id": "47",
            "cluster": "79",
            "cluster_label": "79; Citalopram Depression Treatment",
            "title": "Citalopram therapy for depression: A review of 10 years of european experience and data from U.S. clinical trials",
        },
        "position": {"x": 412.5, "y": 2015},
    },
    {
        "data": {
            "id": "48",
            "cluster": "79",
            "cluster_label": "79; Citalopram Depression Treatment",
            "title": "Lack of effect of a single dose of ketoconazole on the pharmacokinetics of citalopram",
        },
        "position": {"x": 542.5, "y": 1935},
    },
    {
        "data": {
            "id": "49",
            "cluster": "64",
            "cluster_label": "64; Clinical Pharmacogenetics; Depression Treatment",
            "title": "Escitalopram (S-citalopram) and its metabolites in vitro: Cytochromes mediating biotransformation, inhibitory effects, and comparison to R-citalopram",
        },
        "position": {"x": 542.5, "y": 2015},
    },
    {
        "data": {
            "id": "50",
            "cluster": "66",
            "cluster_label": "66; Escitalopram Depression Treatment",
            "title": "Escitalopram 10 mg/day is effective and well tolerated in a placebo-controlled study in depression in primary care",
        },
        "position": {"x": 542.5, "y": 2095},
    },
    {
        "data": {
            "id": "51",
            "cluster": "66",
            "cluster_label": "66; Escitalopram Depression Treatment",
            "title": "Escitalopram: A review of its use in the management of major depressive and anxiety disorders",
        },
        "position": {"x": 787.5, "y": 2335},
    },
    {
        "data": {
            "id": "52",
            "cluster": "30",
            "cluster_label": "30; Fear; Animal Studies",
            "title": "Stress-induced vocalisation in adult animals. A valid model of anxiety?",
        },
        "position": {"x": 515, "y": 2175},
    },
    {
        "data": {
            "id": "53",
            "cluster": "66",
            "cluster_label": "66; Escitalopram Depression Treatment",
            "title": "Escitalopram, the S-(+)-enantiomer of citalopram, is a selective serotonin reuptake inhibitor with potent effects in animal models predictive of antidepressant and anxiolytic activities",
        },
        "position": {"x": 552.5, "y": 2255},
    },
    {
        "data": {
            "id": "54",
            "cluster": "66",
            "cluster_label": "66; Escitalopram Depression Treatment",
            "title": "Escitalopram (10-20 mg/day) is effective and well tolerated in a placebo-controlled study in depression in primary care",
        },
        "position": {"x": 657.5, "y": 2335},
    },
    {
        "data": {
            "id": "55",
            "cluster": "66",
            "cluster_label": "66; Escitalopram Depression Treatment",
            "title": "R-citalopram attenuates anxiolytic effects of escitalopram in a rat ultrasonic vocalisation model",
        },
        "position": {"x": 327.5, "y": 2335},
    },
    {
        "data": {
            "id": "56",
            "cluster": "66",
            "cluster_label": "66; Escitalopram Depression Treatment",
            "title": "The R-enantiomer of citalopram counteracts escitalopram-induced increase in extracellular 5-HT in the frontal cortex of freely moving rats",
        },
        "position": {"x": 160, "y": 2255},
    },
    {
        "data": {
            "id": "57",
            "cluster": "66",
            "cluster_label": "66; Escitalopram Depression Treatment",
            "title": "R-citalopram counteracts the effect of escitalopram in a rat conditioned fear stress model of anxiety",
        },
        "position": {"x": 197.5, "y": 2335},
    },
    {
        "data": {
            "id": "58",
            "cluster": "66",
            "cluster_label": "66; Escitalopram Depression Treatment",
            "title": "Comparison of escitalopram and citalopram efficacy: A meta-analysis",
        },
        "position": {"x": 477.5, "y": 2415},
    },
    {
        "data": {
            "id": "59",
            "cluster": "73",
            "cluster_label": "73; Health Economics",
            "title": "Escitalopram: A Pharmacoeconomic Review of its Use in Depression",
        },
        "position": {"x": 617.5, "y": 2495},
    },
    {
        "data": {
            "id": "60",
            "cluster": "66",
            "cluster_label": "66; Escitalopram Depression Treatment",
            "title": "Do equivalent doses of escitalopram and citalopram have similar efficacy? A pooled analysis of two positive placebo-controlled studies in major depressive disorder",
        },
        "position": {"x": 692.5, "y": 2415},
    },
    {
        "data": {
            "id": "61",
            "cluster": "66",
            "cluster_label": "66; Escitalopram Depression Treatment",
            "title": "R-citalopram inhibits functional and 5-HTP-evoked behavioural responses to the SSRI, escitalopram",
        },
        "position": {"x": 207.5, "y": 2415},
    },
    {
        "data": {
            "id": "62",
            "cluster": "66",
            "cluster_label": "66; Escitalopram Depression Treatment",
            "title": "Escitalopram versus citalopram: The surprising role of the R-enantiomer",
        },
        "position": {"x": 477.5, "y": 2495},
    },
    {
        "data": {
            "id": "63",
            "cluster": "66",
            "cluster_label": "66; Escitalopram Depression Treatment",
            "title": "A randomised study comparing escitalopram with venlafaxine XR in primary care patients with major depressive disorder",
        },
        "position": {"x": 310, "y": 2495},
    },
    {
        "data": {
            "id": "64",
            "cluster": "4",
            "cluster_label": "4; Serotonin Regulation Mechanisms; Animal Models",
            "title": "Effects of acute and long-term administration of escitalopram and citalopram on serotonin neurotransmission: An in vivo electrophysiological study in rat brain",
        },
        "position": {"x": 487.5, "y": 2655},
    },
    {
        "data": {
            "id": "65",
            "cluster": "73",
            "cluster_label": "73; Health Economics",
            "title": "Cost-effectiveness of escitalopram versus citalopram in the treatment of severe depression",
        },
        "position": {"x": 617.5, "y": 2575},
    },
    {
        "data": {
            "id": "66",
            "cluster": "66",
            "cluster_label": "66; Escitalopram Depression Treatment",
            "title": "Evidence based review of escitalopram in treating major depressive disorder in primary care",
        },
        "position": {"x": 310, "y": 2575},
    },
    {
        "data": {
            "id": "67",
            "cluster": "66",
            "cluster_label": "66; Escitalopram Depression Treatment",
            "title": "Prospective, multicentre, randomized, double-blind study of the efficacy of escitalopram versus citalopram in outpatient treatment of major depressive disorder",
        },
        "position": {"x": 347.5, "y": 2655},
    },
    {
        "data": {
            "id": "68",
            "cluster": "73",
            "cluster_label": "73; Health Economics",
            "title": "A probabilistic cost-effectiveness analysis of escitalopram, generic citalopram and venlafaxine as a first-line treatment of major depressive disorder in the UK",
        },
        "position": {"x": 617.5, "y": 2655},
    },
    {
        "data": {
            "id": "69",
            "cluster": "66",
            "cluster_label": "66; Escitalopram Depression Treatment",
            "title": "Escitalopram: A review of its use in the management of major depressive disorder",
        },
        "position": {"x": 542.5, "y": 2735},
    },
    {
        "data": {
            "id": "70",
            "cluster": "66",
            "cluster_label": "66; Escitalopram Depression Treatment",
            "title": "Managing depressive and anxiety disorders with escitalopram",
        },
        "position": {"x": 542.5, "y": 2815},
    },
    {
        "data": {
            "id": "71",
            "cluster": "66",
            "cluster_label": "66; Escitalopram Depression Treatment",
            "title": "Efficacy of escitalopram in the treatment of major depressive disorder compared with conventional selective serotonin reuptake inhibitors and venlafaxine XR: A meta-analysis",
        },
        "position": {"x": 337.5, "y": 2815},
    },
    {
        "data": {
            "id": "72",
            "cluster": "66",
            "cluster_label": "66; Escitalopram Depression Treatment",
            "title": "A comparative study of the efficacy of long-term treatment with escitalopram and paroxetine in severely depressed patients",
        },
        "position": {"x": 542.5, "y": 2895},
    },
    {
        "data": {
            "id": "73",
            "cluster": "66",
            "cluster_label": "66; Escitalopram Depression Treatment",
            "title": "Escitalopram versus venlafaxine XR in the treatment of depression",
        },
        "position": {"x": 747.5, "y": 2975},
    },
    {
        "data": {
            "id": "74",
            "cluster": "66",
            "cluster_label": "66; Escitalopram Depression Treatment",
            "title": "A comparative study of the efficacy of acute and continuation treatment with escitalopram versus duloxetine in patients with major depressive disorder",
        },
        "position": {"x": 747.5, "y": 3055},
    },
    {
        "data": {
            "id": "75",
            "cluster": "60",
            "cluster_label": "60; Venlafaxine; Depression Treatment",
            "title": "Comparative efficacy and acceptability of 12 new-generation antidepressants: a multiple-treatments meta-analysis",
        },
        "position": {"x": 562.5, "y": 3215},
    },
    {
        "data": {
            "id": "76",
            "cluster": "66",
            "cluster_label": "66; Escitalopram Depression Treatment",
            "title": "Escitalopram and duloxetine in the treatment of major depressive disorder: A pooled analysis of two trials",
        },
        "position": {"x": 952.5, "y": 3135},
    },
    {
        "data": {
            "id": "77",
            "cluster": "66",
            "cluster_label": "66; Escitalopram Depression Treatment",
            "title": "Escitalopram in the treatment of major depressive disorder: A meta-analysis",
        },
        "position": {"x": 952.5, "y": 3215},
    },
    {
        "data": {
            "id": "78",
            "cluster": "116",
            "cluster_label": "116; Agomelatine Depression Treatment",
            "title": "Agomelatine in the treatment of major depressive disorder: Potential for clinical effectiveness",
        },
        "position": {"x": 747.5, "y": 3295},
    },
    {
        "data": {
            "id": "79",
            "cluster": "116",
            "cluster_label": "116; Agomelatine Depression Treatment",
            "title": "Superior antidepressant efficacy results of agomelatine versus fluoxetine in severe MDD patients: A randomized, double-blind study",
        },
        "position": {"x": 857.5, "y": 3375},
    },
    {
        "data": {
            "id": "80",
            "cluster": "116",
            "cluster_label": "116; Agomelatine Depression Treatment",
            "title": "Novel melatonin-based therapies: Potential advances in the treatment of major depression",
        },
        "position": {"x": 747.5, "y": 3455},
    },
    {
        "data": {
            "id": "81",
            "cluster": "66",
            "cluster_label": "66; Escitalopram Depression Treatment",
            "title": "Escitalopram for the treatment of major depression and anxiety disorders",
        },
        "position": {"x": 692.5, "y": 3135},
    },
    {
        "data": {
            "id": "82",
            "cluster": "13",
            "cluster_label": "13; Pediatric Use, Special Population",
            "title": "Pharmacogenetics of selective serotonin reuptake inhibitors in pediatric depression and anxiety",
        },
        "position": {"x": 692.5, "y": 3215},
    },
    {
        "data": {
            "id": "83",
            "cluster": "1",
            "cluster_label": "1; Pregnancy",
            "title": "Long-term outcome following selective serotonin reuptake inhibitor induced neonatal abstinence syndrome",
        },
        "position": {"x": 617.5, "y": 3295},
    },
    {
        "data": {
            "id": "84",
            "cluster": "1",
            "cluster_label": "1; Pregnancy",
            "title": "Maternal use of SSRIs, SNRIs and NaSSAs: Practical recommendations during pregnancy and lactation",
        },
        "position": {"x": 617.5, "y": 3375},
    },
    {
        "data": {
            "id": "85",
            "cluster": "19",
            "cluster_label": "19; Paroxetine Depression Treatment",
            "title": "Paroxetine for the treatment of depression: A critical update",
        },
        "position": {"x": 617.5, "y": 3455},
    },
    {
        "data": {
            "id": "86",
            "cluster": "66",
            "cluster_label": "66; Escitalopram Depression Treatment",
            "title": "Baseline severity of depression predicts antidepressant drug response relative to escitalopram",
        },
        "position": {"x": 952.5, "y": 3615},
    },
    {
        "data": {
            "id": "87",
            "cluster": "Family",
            "cluster_label": "54;19;54; Fluvoxamine Depression Treatment ::: Paroxetine Depression Treatment ::: Fluvoxamine Depression Treatment",
            "title": "Fluvoxamine versus other anti-depressive agents for depression;Paroxetine versus other anti-depressive agents for depression;Fluoxetine versus other types of pharmacotherapy for depression",
        },
        "position": {"x": 617.5, "y": 3535},
    },
    {
        "data": {
            "id": "88",
            "cluster": "110",
            "cluster_label": "110; Headache Treatment",
            "title": "Selective Serotonin Re-uptake Inhibitors (SSRIs) for preventing migraine and tension-type headaches",
        },
        "position": {"x": 617.5, "y": 3615},
    },
    {
        "data": {
            "id": "89",
            "cluster": "27",
            "cluster_label": "27; Pharmacogenomics;Gene Expression; Neurology",
            "title": "Proteomic investigation of the ventral rat hippocampus links DRP-2 to escitalopram treatment resistance and SNAP to stress resilience in the chronic mild stress model of depression",
        },
        "position": {"x": 365, "y": 3215},
    },
    {
        "data": {
            "id": "90",
            "cluster": "3",
            "cluster_label": "3; Animial Models; Chronic Unpredictable Mild Stress In Mice",
            "title": "Molecular pathways associated with stress resilience and drug resistance in the chronic mild stress rat model of depression - A gene expression study",
        },
        "position": {"x": 365, "y": 3295},
    },
    {
        "data": {
            "id": "91",
            "cluster": "29",
            "cluster_label": "29; Pharmacogenomics; Neurogenic Mechanisms; Animal Models",
            "title": "Escitalopram reduces increased hippocampal cytogenesis in a genetic rat depression model",
        },
        "position": {"x": 365, "y": 3375},
    },
    {
        "data": {
            "id": "92",
            "cluster": "1",
            "cluster_label": "1; Pregnancy",
            "title": "Use of escitalopram during pregnancy: Navigating towards international guidelines and the real world",
        },
        "position": {"x": 617.5, "y": 3695},
    },
    {
        "data": {
            "id": "93",
            "cluster": "29",
            "cluster_label": "29; Pharmacogenomics; Neurogenic Mechanisms; Animal Models",
            "title": "Nortriptyline mediates behavioral effects without affecting hippocampal cytogenesis in a genetic rat depression model",
        },
        "position": {"x": 365, "y": 3455},
    },
    {
        "data": {
            "id": "94",
            "cluster": "29",
            "cluster_label": "29; Pharmacogenomics; Neurogenic Mechanisms; Animal Models",
            "title": "Reversal of hippocampal neuronal maturation by serotonergic antidepressants",
        },
        "position": {"x": 365, "y": 3535},
    },
    {
        "data": {
            "id": "95",
            "cluster": "8",
            "cluster_label": "8; Neonatology; Animal Models,",
            "title": "The age-dependent effects of selective serotonin reuptake inhibitors in humans and rodents: A review",
        },
        "position": {"x": 365, "y": 3615},
    },
    {
        "data": {
            "id": "96",
            "cluster": "8",
            "cluster_label": "8; Neonatology; Animal Models,",
            "title": "Fluoxetine administration to pregnant rats increases anxiety-related behavior in the offspring",
        },
        "position": {"x": 365, "y": 3695},
    },
    {
        "data": {
            "id": "97",
            "cluster": "8",
            "cluster_label": "8; Neonatology; Animal Models,",
            "title": "Fluoxetine during development reverses the effects of prenatal stress on depressive-like behavior and hippocampal neurogenesis in adolescence",
        },
        "position": {"x": 365, "y": 3775},
    },
    {
        "data": {
            "id": "98",
            "cluster": "1",
            "cluster_label": "1; Pregnancy",
            "title": "Drug treatment for mood disorders in pregnancy",
        },
        "position": {"x": 617.5, "y": 3775},
    },
    {
        "data": {
            "id": "99",
            "cluster": "1",
            "cluster_label": "1; Pregnancy",
            "title": "Selective serotonin reuptake inhibitor exposure during early pregnancy and the risk of birth defects",
        },
        "position": {"x": 822.5, "y": 3855},
    },
    {
        "data": {
            "id": "100",
            "cluster": "66",
            "cluster_label": "66; Escitalopram Depression Treatment",
            "title": "Comparative efficacy of escitalopram in the treatment of major depressive disorder",
        },
        "position": {"x": 952.5, "y": 3695},
    },
    {
        "data": {
            "id": "101",
            "cluster": "8",
            "cluster_label": "8; Neonatology; Animal Models,",
            "title": "Chronic fluoxetine treatment and maternal adversity differentially alter neurobehavioral outcomes in the rat dam",
        },
        "position": {"x": 365, "y": 3855},
    },
    {
        "data": {
            "id": "102",
            "cluster": "8",
            "cluster_label": "8; Neonatology; Animal Models,",
            "title": "Developmental fluoxetine exposure differentially alters central and peripheral measures of the HPA system in adolescent male and female offspring",
        },
        "position": {"x": 552.5, "y": 3935},
    },
    {
        "data": {
            "id": "103",
            "cluster": "1",
            "cluster_label": "1; Pregnancy",
            "title": "Maternal use of selective serotonin reuptake inhibitors, fetal growth, and risk of adverse birth outcomes",
        },
        "position": {"x": 822.5, "y": 3935},
    },
    {
        "data": {
            "id": "104",
            "cluster": "66",
            "cluster_label": "66; Escitalopram Depression Treatment",
            "title": "Escitalopram, an antidepressant with an allosteric effect at the serotonin transporter-a review of current understanding of its mechanism of action",
        },
        "position": {"x": 952.5, "y": 3775},
    },
    {
        "data": {
            "id": "105",
            "cluster": "2",
            "cluster_label": "2; Pharmaceutical Chemistry; Antidepressant Detection",
            "title": "A validated microfluidics-based LC-chip-MS/MS method for the quantitation of fluoxetine and norfluoxetine in rat serum",
        },
        "position": {"x": 402.5, "y": 3935},
    },
    {
        "data": {
            "id": "106",
            "cluster": "8",
            "cluster_label": "8; Neonatology; Animal Models,",
            "title": "Behavioural outcomes of perinatal maternal fluoxetine treatment",
        },
        "position": {"x": 692.5, "y": 4015},
    },
    {
        "data": {
            "id": "107",
            "cluster": "1",
            "cluster_label": "1; Pregnancy",
            "title": "Parental depression, maternal antidepressant use during pregnancy, and risk of autism spectrum disorders: Population based case-control study",
        },
        "position": {"x": 822.5, "y": 4015},
    },
    {
        "data": {
            "id": "108",
            "cluster": "1",
            "cluster_label": "1; Pregnancy",
            "title": "Use of selective serotonin reuptake inhibitors during pregnancy and risk of autism",
        },
        "position": {"x": 822.5, "y": 4095},
    },
    {
        "data": {
            "id": "109",
            "cluster": "8",
            "cluster_label": "8; Neonatology; Animal Models,",
            "title": "Developmental fluoxetine exposure and prenatal stress alter sexual differentiation of the brain and reproductive behavior in male rat offspring",
        },
        "position": {"x": 562.5, "y": 4015},
    },
    {
        "data": {
            "id": "110",
            "cluster": "8",
            "cluster_label": "8; Neonatology; Animal Models,",
            "title": "Developmental Fluoxetine Exposure Normalizes the Long-Term Effects of Maternal Stress on Post-Operative Pain in Sprague-Dawley Rat Offspring",
        },
        "position": {"x": 365, "y": 4015},
    },
    {
        "data": {
            "id": "111",
            "cluster": "8",
            "cluster_label": "8; Neonatology; Animal Models,",
            "title": "Long-term outcomes of developmental exposure to fluoxetine: A review of the animal literature",
        },
        "position": {"x": 552.5, "y": 4095},
    },
    {
        "data": {
            "id": "112",
            "cluster": "24",
            "cluster_label": "24; Pychopharmacology; Reviews",
            "title": "Serotonergic drugs for depression and beyond",
        },
        "position": {"x": 952.5, "y": 3855},
    },
    {
        "data": {
            "id": "113",
            "cluster": "6",
            "cluster_label": "6; Aquatic Toxicology",
            "title": "Similar anxiolytic effects of agonists targeting serotonin 5-HT<inf>1A</inf> or cannabinoid CB receptors on zebrafish behavior in novel environments",
        },
        "position": {"x": 952.5, "y": 3935},
    },
    {
        "data": {
            "id": "114",
            "cluster": "6",
            "cluster_label": "6; Aquatic Toxicology",
            "title": "Fish on Prozac (and Zoloft): Ten years later",
        },
        "position": {"x": 952.5, "y": 4015},
    },
    {
        "data": {
            "id": "115",
            "cluster": "8",
            "cluster_label": "8; Neonatology; Animal Models,",
            "title": "Developmental fluoxetine exposure facilitates sexual behavior in female offspring",
        },
        "position": {"x": 365, "y": 4095},
    },
    {
        "data": {
            "id": "116",
            "cluster": "6",
            "cluster_label": "6; Aquatic Toxicology",
            "title": "Bioaccumulation and trophic dilution of human pharmaceuticals across trophic positions of an effluent-dependent wadeable stream",
        },
        "position": {"x": 952.5, "y": 4095},
    },
    {
        "data": {
            "id": "117",
            "cluster": "1",
            "cluster_label": "1; Pregnancy",
            "title": "Prenatal SSRI use and offspring with autism spectrum disorder or developmental delay",
        },
        "position": {"x": 822.5, "y": 4175},
    },
    {
        "data": {
            "id": "118",
            "cluster": "8",
            "cluster_label": "8; Neonatology; Animal Models,",
            "title": "Increased aggression, improved spatial memory, and reduced anxiety-like behaviour in adult male mice exposed to fluoxetine early in life",
        },
        "position": {"x": 532.5, "y": 4175},
    },
    {
        "data": {
            "id": "119",
            "cluster": "1",
            "cluster_label": "1; Pregnancy",
            "title": "Exposure to selective serotonin reuptake inhibitors during pregnancy and risk of autism spectrum disorder in children: A systematic review and meta-analysis of observational studies",
        },
        "position": {"x": 822.5, "y": 4255},
    },
    {
        "data": {
            "id": "120",
            "cluster": "1",
            "cluster_label": "1; Pregnancy",
            "title": "Prenatal antidepressant exposure and the risk of autism spectrum disorders in children. Are we looking at the fall of Gods?",
        },
        "position": {"x": 1120, "y": 4335},
    },
    {
        "data": {
            "id": "121",
            "cluster": "8",
            "cluster_label": "8; Neonatology; Animal Models,",
            "title": "Developmental exposure to SSRIs, in addition to maternal stress, has long-term sex-dependent effects on hippocampal plasticity",
        },
        "position": {"x": 365, "y": 4175},
    },
    {
        "data": {
            "id": "122",
            "cluster": "27",
            "cluster_label": "27; Pharmacogenomics;Gene Expression; Neurology",
            "title": "Long-term consequences of chronic fluoxetine exposure on the expression of myelination-related genes in the rat hippocampus",
        },
        "position": {"x": 505, "y": 4255},
    },
    {
        "data": {
            "id": "123",
            "cluster": "6",
            "cluster_label": "6; Aquatic Toxicology",
            "title": "Behavioral and biochemical responses in freshwater fish Carassius auratus exposed to sertraline",
        },
        "position": {"x": 952.5, "y": 4175},
    },
    {
        "data": {
            "id": "124",
            "cluster": "6",
            "cluster_label": "6; Aquatic Toxicology",
            "title": "Assessing potential vulnerability and response of fish to simulated avian predation after exposure to psychotropic pharmaceuticals",
        },
        "position": {"x": 952.5, "y": 4255},
    },
    {
        "data": {
            "id": "125",
            "cluster": "8",
            "cluster_label": "8; Neonatology; Animal Models,",
            "title": "Prenatal exposure to selective serotonin reuptake inhibitors (SSRI) increases aggression and modulates maternal behavior in offspring mice",
        },
        "position": {"x": 860, "y": 4335},
    },
    {
        "data": {
            "id": "126",
            "cluster": "8",
            "cluster_label": "8; Neonatology; Animal Models,",
            "title": "Maternal postpartum corticosterone and fluoxetine differentially affect adult male and female offspring on anxiety-like behavior, stress reactivity, and hippocampal neurogenesis",
        },
        "position": {"x": 365, "y": 4255},
    },
    {
        "data": {
            "id": "127",
            "cluster": "8",
            "cluster_label": "8; Neonatology; Animal Models,",
            "title": "Parity modifies the effects of fluoxetine and corticosterone on behavior, stress reactivity, and hippocampal neurogenesis",
        },
        "position": {"x": 245, "y": 4335},
    },
    {
        "data": {
            "id": "128",
            "cluster": "8",
            "cluster_label": "8; Neonatology; Animal Models,",
            "title": "Gestational stress and fluoxetine treatment differentially affect plasticity, methylation and serotonin levels in the PFC and hippocampus of rat dams",
        },
        "position": {"x": 245, "y": 4415},
    },
    {
        "data": {
            "id": "129",
            "cluster": "6",
            "cluster_label": "6; Aquatic Toxicology",
            "title": "Impacts of the antidepressant fluoxetine on the anti-predator behaviours of wild guppies (Poecilia reticulata)",
        },
        "position": {"x": 655, "y": 4335},
    },
    {
        "data": {
            "id": "130",
            "cluster": "1",
            "cluster_label": "1; Pregnancy",
            "title": "The association between antenatal exposure to selective serotonin reuptake inhibitors and autism: A systematic review and meta-analysis",
        },
        "position": {"x": 1120, "y": 4415},
    },
    {
        "data": {
            "id": "131",
            "cluster": "1",
            "cluster_label": "1; Pregnancy",
            "title": "Association between serotonergic antidepressant use during pregnancy and autism spectrum disorder in children",
        },
        "position": {"x": 1120, "y": 4495},
    },
    {
        "data": {
            "id": "132",
            "cluster": "8",
            "cluster_label": "8; Neonatology; Animal Models,",
            "title": "Increased symptoms of illness following prenatal stress: Can it be prevented by fluoxetine?",
        },
        "position": {"x": 990, "y": 4415},
    },
    {
        "data": {
            "id": "133",
            "cluster": "8",
            "cluster_label": "8; Neonatology; Animal Models,",
            "title": "Prenatal fluoxetine modifies the behavioral and hormonal responses to stress in male mice: Role for glucocorticoid insensitivity",
        },
        "position": {"x": 990, "y": 4495},
    },
    {
        "data": {
            "id": "134",
            "cluster": "8",
            "cluster_label": "8; Neonatology; Animal Models,",
            "title": "Perinatal fluoxetine effects on social play, the HPA system, and hippocampal plasticity in pre-adolescent male and female rats: Interactions with pre-gestational maternal stress",
        },
        "position": {"x": 655, "y": 4575},
    },
    {
        "data": {
            "id": "135",
            "cluster": "6",
            "cluster_label": "6; Aquatic Toxicology",
            "title": "An AOP analysis of selective serotonin reuptake inhibitors (SSRIs) for fish",
        },
        "position": {"x": 830, "y": 4415},
    },
    {
        "data": {
            "id": "136",
            "cluster": "6",
            "cluster_label": "6; Aquatic Toxicology",
            "title": "The antidepressant fluoxetine alters mechanisms of pre- and post-copulatory sexual selection in the eastern mosquitofish (Gambusia holbrooki)",
        },
        "position": {"x": 655, "y": 4415},
    },
    {
        "data": {
            "id": "137",
            "cluster": "8",
            "cluster_label": "8; Neonatology; Animal Models,",
            "title": "Perinatal fluoxetine prevents the effect of pre-gestational maternal stress on 5-HT in the PFC, but maternal stress has enduring effects on mPFC synaptic structure in offspring",
        },
        "position": {"x": 655, "y": 4655},
    },
    {
        "data": {
            "id": "138",
            "cluster": "8",
            "cluster_label": "8; Neonatology; Animal Models,",
            "title": "Perinatal selective serotonin reuptake inhibitor (SSRI) effects on body weight at birth and beyond: A review of animal and human studies",
        },
        "position": {"x": 655, "y": 4735},
    },
    {
        "data": {
            "id": "139",
            "cluster": "8",
            "cluster_label": "8; Neonatology; Animal Models,",
            "title": "Subjecting Dams to Early Life Stress and Perinatal Fluoxetine Treatment Differentially Alters Social Behavior in Young and Adult Rat Offspring",
        },
        "position": {"x": 655, "y": 4815},
    },
    {
        "data": {
            "id": "140",
            "cluster": "8",
            "cluster_label": "8; Neonatology; Animal Models,",
            "title": "Perinatal fluoxetine exposure changes social and stress-coping behavior in adult rats housed in a seminatural environment",
        },
        "position": {"x": 655, "y": 4895},
    },
    {
        "data": {
            "id": "141",
            "cluster": "6",
            "cluster_label": "6; Aquatic Toxicology",
            "title": "The pharmaceutical pollutant fluoxetine alters reproductive behaviour in a fish independent of predation risk",
        },
        "position": {"x": 655, "y": 4495},
    },
    {
        "data": {
            "id": "142",
            "cluster": "6",
            "cluster_label": "6; Aquatic Toxicology",
            "title": "Impact of the widespread pharmaceutical pollutant fluoxetine on behaviour and sperm traits in a freshwater fish",
        },
        "position": {"x": 395, "y": 4575},
    },
    {
        "data": {
            "id": "143",
            "cluster": "Family",
            "cluster_label": "7;6; Environmental Science; Wastewater ::: Aquatic Toxicology",
            "title": "Global scanning of selective serotonin reuptake inhibitors: occurrence, wastewater treatment and hazards in aquatic systems;Antidepressants in Surface Waters: Fluoxetine Influences Mosquitofish Anxiety-Related Behavior at Environmentally Relevant Levels",
        },
        "position": {"x": 395, "y": 4655},
    },
    {
        "data": {
            "id": "144",
            "cluster": "6",
            "cluster_label": "6; Aquatic Toxicology",
            "title": "Field-realistic antidepressant exposure disrupts group foraging dynamics in mosquitofish",
        },
        "position": {"x": 395, "y": 4735},
    },
    {
        "data": {
            "id": "145",
            "cluster": "8",
            "cluster_label": "8; Neonatology; Animal Models,",
            "title": "Perinatal fluoxetine treatment and dams\u2019 early life stress history alter affective behavior in rat offspring depending on serotonin transporter genotype and sex",
        },
        "position": {"x": 655, "y": 4975},
    },
    {
        "data": {
            "id": "146",
            "cluster": "6",
            "cluster_label": "6; Aquatic Toxicology",
            "title": "Chronic exposure to a pervasive pharmaceutical pollutant erodes among-individual phenotypic variation in a fish",
        },
        "position": {"x": 395, "y": 4815},
    },
    {
        "data": {
            "id": "147",
            "cluster": "6",
            "cluster_label": "6; Aquatic Toxicology",
            "title": "Antidepressant exposure causes a nonmonotonic reduction in anxiety-related behaviour in female mosquitofish",
        },
        "position": {"x": 395, "y": 4895},
    },
    {
        "data": {
            "id": "148",
            "cluster": "29",
            "cluster_label": "29; Pharmacogenomics; Neurogenic Mechanisms; Animal Models",
            "title": "Effect of fluoxetine on proliferation and/or survival of microglia and oligodendrocyte progenitor cells in the fornix and corpus callosum of the mouse brain",
        },
        "position": {"x": 375, "y": 4335},
    },
    {
        "data": {
            "id": "149",
            "cluster": "40",
            "cluster_label": "40; Covid-19",
            "title": "Drug repurposing of selective serotonin reuptake inhibitors: Could these drugs help fight COVID-19 and save lives?",
        },
        "position": {"x": 375, "y": 4415},
    },
    {
        "data": {
            "id": "150",
            "cluster": "40",
            "cluster_label": "40; Covid-19",
            "title": "Effect of early treatment with fluvoxamine on risk of emergency care and hospitalisation among patients with COVID-19: the TOGETHER randomised, platform clinical trial",
        },
        "position": {"x": 375, "y": 4495},
    },
    {
        "data": {
            "id": "151",
            "cluster": "8",
            "cluster_label": "8; Neonatology; Animal Models,",
            "title": "The combination of fluoxetine and environmental enrichment reduces postpartum stress-related behaviors through the oxytocinergic system and HPA axis in mice",
        },
        "position": {"x": 655, "y": 5055},
    },
    {
        "data": {
            "id": "152",
            "cluster": "8",
            "cluster_label": "8; Neonatology; Animal Models,",
            "title": "Perinatal fluoxetine dose-dependently affects prenatal stress-induced neurobehavioural abnormalities, HPA-axis functioning and underlying brain alterations in rat dams and their offspring",
        },
        "position": {"x": 655, "y": 5135},
    },
    {
        "data": {
            "id": "153",
            "cluster": "8",
            "cluster_label": "8; Neonatology; Animal Models,",
            "title": "Sex differences in long-term behavioral alterations, especially anxiety, following prenatal fluoxetine exposure in C57BL/6 mice",
        },
        "position": {"x": 655, "y": 5215},
    },
    {
        "data": {
            "id": "154",
            "cluster": "6",
            "cluster_label": "6; Aquatic Toxicology",
            "title": "Context is Key: Social Environment Mediates the Impacts of a Psychoactive Pollutant on Shoaling Behavior in Fish",
        },
        "position": {"x": 395, "y": 4975},
    },
    {
        "data": {
            "id": "155",
            "cluster": "40",
            "cluster_label": "40; Covid-19",
            "title": "Safety and efficacy of fluvoxamine in COVID-19 ICU patients: An open label, prospective cohort trial with matched controls",
        },
        "position": {"x": 50, "y": 4575},
    },
    {
        "data": {
            "id": "156",
            "cluster": "40",
            "cluster_label": "40; Covid-19",
            "title": "Mechanisms of action of fluvoxamine for COVID-19: a historical review",
        },
        "position": {"x": 50, "y": 4655},
    },
    {
        "data": {
            "id": "157",
            "cluster": "8",
            "cluster_label": "8; Neonatology; Animal Models,",
            "title": "Prenatal exposure to fluoxetine modulates emotionality and aversive memory in male and female rat offspring",
        },
        "position": {"x": 655, "y": 5295},
    },
    {
        "data": {
            "id": "158",
            "cluster": "40",
            "cluster_label": "40; Covid-19",
            "title": "Association between antidepressant use and ED or hospital visits in outpatients with SARS-CoV-2",
        },
        "position": {"x": 50, "y": 4735},
    },
    {
        "data": {
            "id": "159",
            "cluster": "40",
            "cluster_label": "40; Covid-19",
            "title": "Antidepressant Use and Its Association with 28-Day Mortality in Inpatients with SARS-CoV-2: Support for the FIASMA Model against COVID-19",
        },
        "position": {"x": 50, "y": 4815},
    },
    {
        "data": {
            "id": "160",
            "cluster": "40",
            "cluster_label": "40; Covid-19",
            "title": "Antiviral and Anti-Inflammatory Activities of Fluoxetine in a SARS-CoV-2 Infection Mouse Model",
        },
        "position": {"x": 50, "y": 4895},
    },
    {
        "data": {
            "id": "161",
            "cluster": "6",
            "cluster_label": "6; Aquatic Toxicology",
            "title": "Effects of fluoxetine on fish: What do we know and where should we focus our efforts in the future?",
        },
        "position": {"x": 395, "y": 5055},
    },
    {
        "data": {
            "id": "162",
            "cluster": "6",
            "cluster_label": "6; Aquatic Toxicology",
            "title": "Identifying knowledge gaps in understanding the effects of selective serotonin reuptake inhibitors (SSRIs) on fish behaviour",
        },
        "position": {"x": 395, "y": 5135},
    },
    {
        "data": {
            "id": "163",
            "cluster": "6",
            "cluster_label": "6; Aquatic Toxicology",
            "title": "Sex-specific effects of psychoactive pollution on behavioral individuality and plasticity in fish",
        },
        "position": {"x": 525, "y": 5215},
    },
    {
        "data": {
            "id": "164",
            "cluster": "40",
            "cluster_label": "40; Covid-19",
            "title": "Association of fluvoxamine with mortality and symptom resolution among inpatients with COVID-19 in Uganda: a prospective interventional open-label cohort study",
        },
        "position": {"x": 50, "y": 4975},
    },
    {
        "data": {
            "id": "165",
            "cluster": "6",
            "cluster_label": "6; Aquatic Toxicology",
            "title": "Are early and young life stages of fish affected by paroxetine? A case study with Danio rerio",
        },
        "position": {"x": 395, "y": 5215},
    },
    {
        "data": {
            "id": "166",
            "cluster": "8",
            "cluster_label": "8; Neonatology; Animal Models,",
            "title": "Perinatal serotonergic manipulation shapes anhedonic and cognitive behaviors in a sex- and age-dependent manner: Identification of related biological functions at central and peripheral level",
        },
        "position": {"x": 655, "y": 5375},
    },
    {
        "data": {
            "id": "167",
            "cluster": "40",
            "cluster_label": "40; Covid-19",
            "title": "Overview of the potential use of fluvoxamine for COVID-19 and long COVID",
        },
        "position": {"x": 50, "y": 5055},
    },
    {
        "data": {
            "id": "168",
            "cluster": "40",
            "cluster_label": "40; Covid-19",
            "title": "Detrimental effects of COVID-19 in the brain and therapeutic options for long COVID: The role of Epstein\u2013Barr virus and the gut\u2013brain axis",
        },
        "position": {"x": 170, "y": 5135},
    },
    {
        "data": {
            "id": "169",
            "cluster": "40",
            "cluster_label": "40; Covid-19",
            "title": "Evaluating fluvoxamine for the outpatient treatment of COVID-19: A systematic review and meta-analysis",
        },
        "position": {"x": 40, "y": 5135},
    },
    {
        "data": {
            "source": "1",
            "target": "2",
            "id": "53176966-675d-4d51-8dc9-7817f7b84259",
        }
    },
    {
        "data": {
            "source": "2",
            "target": "4",
            "id": "6cfe6aef-bfe9-4e6c-be8a-a0506f7e162b",
        }
    },
    {
        "data": {
            "source": "3",
            "target": "6",
            "id": "2271453c-b7cd-4c62-8603-19ad8bccd947",
        }
    },
    {
        "data": {
            "source": "6",
            "target": "7",
            "id": "430c0d0c-2f91-435a-a929-17b84d517fa0",
        }
    },
    {
        "data": {
            "source": "3",
            "target": "8",
            "id": "2454026f-9fa6-4611-b802-5761e901e043",
        }
    },
    {
        "data": {
            "source": "5",
            "target": "8",
            "id": "1f67c8ce-8df0-4ac4-b634-256bb6ab3af0",
        }
    },
    {
        "data": {
            "source": "8",
            "target": "9",
            "id": "b3f7d6e6-b57d-4e99-9717-72b4f7b87387",
        }
    },
    {
        "data": {
            "source": "4",
            "target": "10",
            "id": "db50a70b-34d0-42b6-ae82-11ffdcaeadbe",
        }
    },
    {
        "data": {
            "source": "10",
            "target": "11",
            "id": "4eba396f-9c33-4e3e-9b76-d0e00eb6fc7e",
        }
    },
    {
        "data": {
            "source": "7",
            "target": "12",
            "id": "087f5ada-91be-49e4-a7fc-c4292b1f4a02",
        }
    },
    {
        "data": {
            "source": "11",
            "target": "13",
            "id": "1666f6ec-63aa-4b8b-8d05-18b13ea39ba6",
        }
    },
    {
        "data": {
            "source": "12",
            "target": "14",
            "id": "494354f8-f045-4745-a120-91232cff3fb0",
        }
    },
    {
        "data": {
            "source": "9",
            "target": "14",
            "id": "f9422a36-5b37-45ba-8d06-24250c0b21bc",
        }
    },
    {
        "data": {
            "source": "13",
            "target": "15",
            "id": "0fb252e6-ef97-4d71-9031-be1a9494b08c",
        }
    },
    {
        "data": {
            "source": "15",
            "target": "16",
            "id": "121d710d-4765-4b54-a012-0637b7e6edbf",
        }
    },
    {
        "data": {
            "source": "16",
            "target": "17",
            "id": "7ae2d6f3-cf55-4530-9e7f-5ca1d3428e73",
        }
    },
    {
        "data": {
            "source": "14",
            "target": "18",
            "id": "d93f5d86-61f9-469e-9354-56c6a928a8e3",
        }
    },
    {
        "data": {
            "source": "18",
            "target": "19",
            "id": "0e28d0cb-da4f-4001-8ba8-3e31ee4edfcc",
        }
    },
    {
        "data": {
            "source": "19",
            "target": "20",
            "id": "6199e5ed-f3ed-43ef-86ed-35123ea52803",
        }
    },
    {
        "data": {
            "source": "20",
            "target": "21",
            "id": "63f113f7-8f0f-4473-a76b-5ae71a9230d5",
        }
    },
    {
        "data": {
            "source": "21",
            "target": "22",
            "id": "e420e4eb-c1b7-4198-9be6-48fc48696d50",
        }
    },
    {
        "data": {
            "source": "17",
            "target": "22",
            "id": "29e6c1a3-03de-4b90-9cfb-a2700267477d",
        }
    },
    {
        "data": {
            "source": "20",
            "target": "22",
            "id": "1e1b7b6d-8448-4c0f-bcc4-8cee841e72da",
        }
    },
    {
        "data": {
            "source": "22",
            "target": "23",
            "id": "1dae0a7e-c012-467b-8f3a-17ade72e1b10",
        }
    },
    {
        "data": {
            "source": "23",
            "target": "24",
            "id": "0290f5e6-e21e-4f33-830d-27820616fac3",
        }
    },
    {
        "data": {
            "source": "24",
            "target": "25",
            "id": "e7fc7d3d-e188-421e-a965-f6143c315491",
        }
    },
    {
        "data": {
            "source": "25",
            "target": "26",
            "id": "a866bdbb-19c1-4356-8786-3a41a2edeb1e",
        }
    },
    {
        "data": {
            "source": "26",
            "target": "27",
            "id": "99fa13ce-5166-44ed-9138-a808ec5ebdf7",
        }
    },
    {
        "data": {
            "source": "25",
            "target": "28",
            "id": "ae18ab40-ff07-417e-8cd9-d8b0f058062f",
        }
    },
    {
        "data": {
            "source": "28",
            "target": "29",
            "id": "7ddfc8ed-4488-4df4-944a-74c33bd282d6",
        }
    },
    {
        "data": {
            "source": "25",
            "target": "30",
            "id": "7e1798ab-4ad4-4c4f-a420-d61d31b8282f",
        }
    },
    {
        "data": {
            "source": "24",
            "target": "30",
            "id": "bf57bef9-bd73-41b4-b47f-1d24bed6e062",
        }
    },
    {
        "data": {
            "source": "30",
            "target": "31",
            "id": "26e8057c-5d37-4f12-a4cf-f63ba0bc134f",
        }
    },
    {
        "data": {
            "source": "31",
            "target": "32",
            "id": "d8d430b3-0a05-4ede-85f5-79d8827f1f16",
        }
    },
    {
        "data": {
            "source": "29",
            "target": "33",
            "id": "9d832bda-3f7c-468f-bd3a-8889d092521d",
        }
    },
    {
        "data": {
            "source": "32",
            "target": "34",
            "id": "99375978-ae4c-4a46-a421-f9ab46da2274",
        }
    },
    {
        "data": {
            "source": "30",
            "target": "35",
            "id": "37269535-cb26-4719-ae07-c25f83cfd21c",
        }
    },
    {
        "data": {
            "source": "27",
            "target": "36",
            "id": "8a5c2e32-a29a-44ff-8a13-aba9a310452a",
        }
    },
    {
        "data": {
            "source": "36",
            "target": "37",
            "id": "5a28bb78-eff4-4305-bb8e-8d6a23eaea6c",
        }
    },
    {
        "data": {
            "source": "35",
            "target": "38",
            "id": "a7b640b9-28d7-4ddd-869c-4a63924b7b2e",
        }
    },
    {
        "data": {
            "source": "37",
            "target": "39",
            "id": "3c995308-2538-4c40-b9f0-8cbfd2d5d730",
        }
    },
    {
        "data": {
            "source": "38",
            "target": "39",
            "id": "d164dd55-9ad5-45e1-933c-f0752e9f39c6",
        }
    },
    {
        "data": {
            "source": "39",
            "target": "40",
            "id": "a2ff4fad-0309-429c-bf1c-c76cc30d93a4",
        }
    },
    {
        "data": {
            "source": "34",
            "target": "40",
            "id": "9c9a7e03-d93d-4e2c-a762-d75b95c04d44",
        }
    },
    {
        "data": {
            "source": "33",
            "target": "40",
            "id": "d26f6c06-648f-4d87-ac48-e8c59222341a",
        }
    },
    {
        "data": {
            "source": "40",
            "target": "41",
            "id": "8ab1be0a-f038-4088-ad60-aea0dda68864",
        }
    },
    {
        "data": {
            "source": "41",
            "target": "42",
            "id": "7ecd4885-0788-4c8c-ac84-215fe3e6ba00",
        }
    },
    {
        "data": {
            "source": "41",
            "target": "43",
            "id": "c3a56b30-67df-46ff-80d6-20d3d6b7fab3",
        }
    },
    {
        "data": {
            "source": "42",
            "target": "44",
            "id": "3dc8380f-7bd8-42df-a361-bd8b40c6e78b",
        }
    },
    {
        "data": {
            "source": "43",
            "target": "44",
            "id": "b808c7fa-4040-459c-a8bd-52a04abd0de4",
        }
    },
    {
        "data": {
            "source": "44",
            "target": "45",
            "id": "3c7e89e2-2779-4b14-b5d8-99c9968951b1",
        }
    },
    {
        "data": {
            "source": "45",
            "target": "46",
            "id": "7441d0b6-45f2-4fc8-ac11-f8b52aea52c5",
        }
    },
    {
        "data": {
            "source": "45",
            "target": "47",
            "id": "cbc7809c-b5be-445a-858d-16589ab08877",
        }
    },
    {
        "data": {
            "source": "46",
            "target": "48",
            "id": "c6629fb0-3c6b-4fdf-a984-4043741dfffa",
        }
    },
    {
        "data": {
            "source": "48",
            "target": "49",
            "id": "f5b9103d-24cf-4a50-8d0e-e850a5385b0c",
        }
    },
    {
        "data": {
            "source": "49",
            "target": "50",
            "id": "d25d789f-5a1a-4add-987c-2f4bc7f90c1e",
        }
    },
    {
        "data": {
            "source": "47",
            "target": "50",
            "id": "3bc10dab-5c11-4a17-9744-f0aa56ef1f82",
        }
    },
    {
        "data": {
            "source": "50",
            "target": "51",
            "id": "0c9e3463-202f-4a54-b810-6f4adfd36d5c",
        }
    },
    {
        "data": {
            "source": "50",
            "target": "52",
            "id": "cc7a3203-f86f-4c7b-b702-a7f1b927d1a4",
        }
    },
    {
        "data": {
            "source": "52",
            "target": "53",
            "id": "7bbd5d96-34d1-441f-8f53-4804b29e36be",
        }
    },
    {
        "data": {
            "source": "53",
            "target": "54",
            "id": "43eeee73-91c0-4432-9d03-9e00f4a07b1a",
        }
    },
    {
        "data": {
            "source": "50",
            "target": "54",
            "id": "fe3ce1fd-2d0c-41dc-b8c0-931f0854a360",
        }
    },
    {
        "data": {
            "source": "52",
            "target": "55",
            "id": "8de919ae-7614-4383-b7ef-09741946d80e",
        }
    },
    {
        "data": {
            "source": "50",
            "target": "56",
            "id": "fe462195-91b2-44e1-a9b2-1ece18560e20",
        }
    },
    {
        "data": {
            "source": "53",
            "target": "57",
            "id": "4b5482d3-c7cd-4728-9f88-91249bdf394a",
        }
    },
    {
        "data": {
            "source": "56",
            "target": "57",
            "id": "6f9f6a65-eb5f-4aa3-a43c-aa28f7778fe2",
        }
    },
    {
        "data": {
            "source": "54",
            "target": "58",
            "id": "b6f66fde-1283-48ed-bfa5-8936615c62c8",
        }
    },
    {
        "data": {
            "source": "51",
            "target": "58",
            "id": "0cc626e4-2e2b-4d82-88af-60f46225c058",
        }
    },
    {
        "data": {
            "source": "53",
            "target": "58",
            "id": "eeb739f2-fe91-4a8c-a97c-2843fbfc7846",
        }
    },
    {
        "data": {
            "source": "55",
            "target": "58",
            "id": "3baff1c1-5152-46f9-ac2c-2d40279b24b5",
        }
    },
    {
        "data": {
            "source": "56",
            "target": "58",
            "id": "b12d7f6b-9397-4851-8c4d-f4482b70a219",
        }
    },
    {
        "data": {
            "source": "58",
            "target": "59",
            "id": "2a18a819-3ac7-4312-ac85-381945c818ce",
        }
    },
    {
        "data": {
            "source": "54",
            "target": "60",
            "id": "7bff68f5-8441-4ce4-9fc4-2e85ba43c99d",
        }
    },
    {
        "data": {
            "source": "57",
            "target": "61",
            "id": "230534ac-a299-4b84-9fe1-96d7bc5ca5be",
        }
    },
    {
        "data": {
            "source": "54",
            "target": "61",
            "id": "7ef90ba9-8740-4f4f-beb3-d0faddaa67e3",
        }
    },
    {
        "data": {
            "source": "61",
            "target": "62",
            "id": "be0d7a67-96c2-492f-9e35-351bf34879f1",
        }
    },
    {
        "data": {
            "source": "60",
            "target": "62",
            "id": "e017eab2-7d07-469e-9835-da22478d9b42",
        }
    },
    {
        "data": {
            "source": "58",
            "target": "62",
            "id": "95eb15d3-6d53-4955-88d0-e4d9b57597f3",
        }
    },
    {
        "data": {
            "source": "58",
            "target": "63",
            "id": "40536b05-d315-4115-bef7-cd7c74b93894",
        }
    },
    {
        "data": {
            "source": "62",
            "target": "64",
            "id": "9217120b-cdf5-49c8-9447-2e32b005f267",
        }
    },
    {
        "data": {
            "source": "59",
            "target": "65",
            "id": "e7ae8bb8-7bf6-4c46-b6d1-f972395f58ef",
        }
    },
    {
        "data": {
            "source": "63",
            "target": "66",
            "id": "6fb20298-d615-430d-ad80-06715b60b620",
        }
    },
    {
        "data": {
            "source": "62",
            "target": "67",
            "id": "4bcfefbf-7cf0-49cc-ab13-d969909e1e51",
        }
    },
    {
        "data": {
            "source": "66",
            "target": "67",
            "id": "365b60ce-f7de-4ff9-b1c4-d14eb4fefbc4",
        }
    },
    {
        "data": {
            "source": "65",
            "target": "68",
            "id": "3abd0907-00dc-44e7-b1fb-a88d744eab24",
        }
    },
    {
        "data": {
            "source": "67",
            "target": "69",
            "id": "5bf08087-bffb-4045-9d18-53b3eee2e6df",
        }
    },
    {
        "data": {
            "source": "64",
            "target": "69",
            "id": "dbe4317c-de8c-49c2-b9b5-1311c074133d",
        }
    },
    {
        "data": {
            "source": "68",
            "target": "69",
            "id": "e6ae6bb1-42a8-4b95-9b43-052192542599",
        }
    },
    {
        "data": {
            "source": "69",
            "target": "70",
            "id": "eef4af5c-132f-499c-97d8-79db6878dc56",
        }
    },
    {
        "data": {
            "source": "67",
            "target": "71",
            "id": "168388a9-1a20-47b3-9390-48b2004cd6ff",
        }
    },
    {
        "data": {
            "source": "70",
            "target": "72",
            "id": "392bda01-78ce-45a9-9912-ecf0af13e6f5",
        }
    },
    {
        "data": {
            "source": "71",
            "target": "72",
            "id": "8ee8f588-c512-409d-9080-a02466f6a131",
        }
    },
    {
        "data": {
            "source": "72",
            "target": "73",
            "id": "278fdfaf-8a80-4e23-92cd-0553768fa841",
        }
    },
    {
        "data": {
            "source": "73",
            "target": "74",
            "id": "c17c5ebd-e873-42c8-93c4-6986b814fab8",
        }
    },
    {
        "data": {
            "source": "72",
            "target": "75",
            "id": "9f270afe-85a9-4077-a530-61c90522b3e9",
        }
    },
    {
        "data": {
            "source": "74",
            "target": "76",
            "id": "85b25116-93d5-4bee-a6a7-2458ef7ecc95",
        }
    },
    {
        "data": {
            "source": "76",
            "target": "77",
            "id": "4e2e3fd9-3841-40e9-94e6-b0ce26ca6125",
        }
    },
    {
        "data": {
            "source": "77",
            "target": "78",
            "id": "4e62e449-2f03-401c-a357-049fead8915f",
        }
    },
    {
        "data": {
            "source": "75",
            "target": "78",
            "id": "cee2de04-d887-4f2a-884e-b72be94bd25a",
        }
    },
    {
        "data": {
            "source": "78",
            "target": "79",
            "id": "48acd8b7-e4cf-476c-97f8-5aca049188dd",
        }
    },
    {
        "data": {
            "source": "79",
            "target": "80",
            "id": "3048f3b9-26cc-4364-9f65-1707d8c2af7f",
        }
    },
    {
        "data": {
            "source": "78",
            "target": "80",
            "id": "4b7d8ee0-abe5-4c36-8cb4-aac478d3dccd",
        }
    },
    {
        "data": {
            "source": "74",
            "target": "81",
            "id": "c0572c25-092a-4b13-a9c0-768f28786356",
        }
    },
    {
        "data": {
            "source": "81",
            "target": "82",
            "id": "602e26f6-c91d-4bd8-8f3c-821fa90504ad",
        }
    },
    {
        "data": {
            "source": "82",
            "target": "83",
            "id": "280b5ee4-fbd8-41bb-9f46-140337b23eb8",
        }
    },
    {
        "data": {
            "source": "83",
            "target": "84",
            "id": "ca36ae64-49e5-4def-930e-5b0c64baea5f",
        }
    },
    {
        "data": {
            "source": "84",
            "target": "85",
            "id": "f95587b6-e73e-441a-a9ef-8629f207712d",
        }
    },
    {
        "data": {
            "source": "77",
            "target": "86",
            "id": "cb049357-1b33-4a43-a6c0-3ef0e3f69778",
        }
    },
    {
        "data": {
            "source": "85",
            "target": "87",
            "id": "ad3df4ea-db1d-4508-b89a-82c472e2fd60",
        }
    },
    {
        "data": {
            "source": "80",
            "target": "87",
            "id": "1a09a359-e406-44a0-a884-fe2fc7872a79",
        }
    },
    {
        "data": {
            "source": "87",
            "target": "88",
            "id": "4f3a47eb-b623-4cd7-9f39-b549d6e64062",
        }
    },
    {
        "data": {
            "source": "72",
            "target": "89",
            "id": "258147c1-421d-4634-9773-8fa0bf6c049c",
        }
    },
    {
        "data": {
            "source": "89",
            "target": "90",
            "id": "f17fd0b8-9136-4f61-9ace-00597ed7af3d",
        }
    },
    {
        "data": {
            "source": "90",
            "target": "91",
            "id": "07a22213-1eb1-40ef-bacf-75cd2b93a9d3",
        }
    },
    {
        "data": {
            "source": "88",
            "target": "92",
            "id": "62cb8245-b82a-4356-b256-4aea5e407d24",
        }
    },
    {
        "data": {
            "source": "91",
            "target": "93",
            "id": "30ae4c2b-4ffc-4ee0-84ee-b333cfdb8098",
        }
    },
    {
        "data": {
            "source": "93",
            "target": "94",
            "id": "37822c7b-1c01-41e1-a9b7-18f8ddcdebf2",
        }
    },
    {
        "data": {
            "source": "94",
            "target": "95",
            "id": "0f0005f2-3bc1-4481-9443-bd3729f2e734",
        }
    },
    {
        "data": {
            "source": "95",
            "target": "96",
            "id": "f4ba2382-fd97-4398-a120-5e205d5224b5",
        }
    },
    {
        "data": {
            "source": "96",
            "target": "97",
            "id": "e2bc751a-758b-4dfe-9f2c-ab4e058e3608",
        }
    },
    {
        "data": {
            "source": "92",
            "target": "98",
            "id": "440bf1e3-8c24-4ec6-a774-bce15ff583f0",
        }
    },
    {
        "data": {
            "source": "98",
            "target": "99",
            "id": "e00bf85e-e4c6-4274-9ef6-36e48a161cb3",
        }
    },
    {
        "data": {
            "source": "86",
            "target": "100",
            "id": "aa737223-a9f0-46b6-9298-68d0ba1aaf01",
        }
    },
    {
        "data": {
            "source": "98",
            "target": "101",
            "id": "74961ce5-85b6-45be-811e-4714f829a2d4",
        }
    },
    {
        "data": {
            "source": "97",
            "target": "101",
            "id": "29262d85-d362-4512-a526-2f8350350538",
        }
    },
    {
        "data": {
            "source": "101",
            "target": "102",
            "id": "2c4add28-98a6-44f9-a0af-bc6262a5bf83",
        }
    },
    {
        "data": {
            "source": "99",
            "target": "103",
            "id": "77b02f72-26a9-4a64-8e64-4083a3ef43e0",
        }
    },
    {
        "data": {
            "source": "100",
            "target": "104",
            "id": "4110be48-4b6d-4859-8c04-22f7124c3a7d",
        }
    },
    {
        "data": {
            "source": "101",
            "target": "105",
            "id": "1176455f-940c-4edf-88a7-9b9b41ae1379",
        }
    },
    {
        "data": {
            "source": "102",
            "target": "106",
            "id": "78c9a90e-c3b0-458d-9af8-080ac8f306c1",
        }
    },
    {
        "data": {
            "source": "103",
            "target": "107",
            "id": "96ff500f-1f4e-442d-8578-5cb1183c8cdc",
        }
    },
    {
        "data": {
            "source": "107",
            "target": "108",
            "id": "85bbc129-041d-4cb4-9c06-7b3b497c48d0",
        }
    },
    {
        "data": {
            "source": "102",
            "target": "109",
            "id": "567bdbd6-b7bf-41cc-8ed2-ee4cc4507c81",
        }
    },
    {
        "data": {
            "source": "102",
            "target": "110",
            "id": "844fba64-3331-4f0e-90e3-9e4a55d080a5",
        }
    },
    {
        "data": {
            "source": "105",
            "target": "110",
            "id": "cc4a4ade-f196-4d59-a5f1-d46336cbb2c9",
        }
    },
    {
        "data": {
            "source": "101",
            "target": "110",
            "id": "404d8780-048a-448e-a4a2-ea7d890810c9",
        }
    },
    {
        "data": {
            "source": "110",
            "target": "111",
            "id": "f397e232-0295-42f7-b988-ad8b12c6a006",
        }
    },
    {
        "data": {
            "source": "106",
            "target": "111",
            "id": "dd8a8564-559c-4b26-bc72-5b3aee4cbdc8",
        }
    },
    {
        "data": {
            "source": "109",
            "target": "111",
            "id": "fda67303-ce62-41c5-82aa-ba763235ab5c",
        }
    },
    {
        "data": {
            "source": "104",
            "target": "112",
            "id": "dbc9016e-7f8d-4c5e-a724-899d28913cea",
        }
    },
    {
        "data": {
            "source": "112",
            "target": "113",
            "id": "9d7c28da-9010-4b16-9494-92740edaa4d3",
        }
    },
    {
        "data": {
            "source": "113",
            "target": "114",
            "id": "2f64548e-e4b7-4eb7-8739-a762dccd8138",
        }
    },
    {
        "data": {
            "source": "110",
            "target": "115",
            "id": "449724a4-42e7-4165-b695-4706c31daba2",
        }
    },
    {
        "data": {
            "source": "114",
            "target": "116",
            "id": "f4a2df24-af63-4490-abd4-eefb3f736f91",
        }
    },
    {
        "data": {
            "source": "108",
            "target": "117",
            "id": "0431e5e2-be49-48a3-87ac-6a726a76c259",
        }
    },
    {
        "data": {
            "source": "111",
            "target": "118",
            "id": "11b4c108-6093-4b8c-8ae7-03f219ec44ac",
        }
    },
    {
        "data": {
            "source": "115",
            "target": "118",
            "id": "10b60d6f-7d24-40c0-897b-87e97ad6216f",
        }
    },
    {
        "data": {
            "source": "117",
            "target": "119",
            "id": "265fadd6-96ff-4cb0-8062-6838bf3a2a08",
        }
    },
    {
        "data": {
            "source": "119",
            "target": "120",
            "id": "41a25d82-5a4d-4e64-b83d-39cd9d20a8ac",
        }
    },
    {
        "data": {
            "source": "115",
            "target": "121",
            "id": "aa0201de-c418-445a-9eaf-c8a713c9e483",
        }
    },
    {
        "data": {
            "source": "121",
            "target": "122",
            "id": "2607b4c0-cc83-4ed3-ae5a-436fef171065",
        }
    },
    {
        "data": {
            "source": "116",
            "target": "123",
            "id": "ae042e66-7a6b-43ec-bd7d-5346eb16a02f",
        }
    },
    {
        "data": {
            "source": "123",
            "target": "124",
            "id": "eda2f977-c37d-4de5-85f4-02442448fce6",
        }
    },
    {
        "data": {
            "source": "118",
            "target": "125",
            "id": "d820c3d8-81bb-4e3e-9eee-b82fa55d1602",
        }
    },
    {
        "data": {
            "source": "118",
            "target": "126",
            "id": "81c4bcc8-5c74-4bc5-b959-3b3a8a49ae05",
        }
    },
    {
        "data": {
            "source": "121",
            "target": "126",
            "id": "5d6700ea-af15-4c38-94f8-32213bf55f5b",
        }
    },
    {
        "data": {
            "source": "126",
            "target": "127",
            "id": "09ac82ac-fe24-44c4-b877-4fead052b975",
        }
    },
    {
        "data": {
            "source": "127",
            "target": "128",
            "id": "bfe10d19-bb76-4ad3-8f9e-33d03d941104",
        }
    },
    {
        "data": {
            "source": "126",
            "target": "129",
            "id": "8a069e57-ace6-4bc4-832e-3084940a8d2e",
        }
    },
    {
        "data": {
            "source": "124",
            "target": "129",
            "id": "286278fb-d212-49b0-8f7c-423203d6938e",
        }
    },
    {
        "data": {
            "source": "120",
            "target": "130",
            "id": "3b88dddb-51b3-4f1f-9675-0e305049622c",
        }
    },
    {
        "data": {
            "source": "130",
            "target": "131",
            "id": "fbfb63db-f4bf-4be4-b477-86905280efa4",
        }
    },
    {
        "data": {
            "source": "125",
            "target": "132",
            "id": "d2343f6f-b103-4b97-b8b2-2bcc4ca0a81f",
        }
    },
    {
        "data": {
            "source": "132",
            "target": "133",
            "id": "15283ba9-a103-418f-a98e-7a774d9a6a49",
        }
    },
    {
        "data": {
            "source": "128",
            "target": "134",
            "id": "7989c482-e18a-4e72-8f09-ec9922900a42",
        }
    },
    {
        "data": {
            "source": "131",
            "target": "134",
            "id": "1cc99866-a3da-4fba-ab70-6a4684b38db1",
        }
    },
    {
        "data": {
            "source": "125",
            "target": "135",
            "id": "f42270f2-544b-4a23-8c03-79b62f6c4ee9",
        }
    },
    {
        "data": {
            "source": "129",
            "target": "136",
            "id": "95bcfddf-ff90-496a-ae11-132819312d11",
        }
    },
    {
        "data": {
            "source": "133",
            "target": "137",
            "id": "7f771ce1-3bd6-455b-9ae3-77109c6eccf8",
        }
    },
    {
        "data": {
            "source": "134",
            "target": "137",
            "id": "5c11e8b9-0138-4383-8629-a3132fc33027",
        }
    },
    {
        "data": {
            "source": "137",
            "target": "138",
            "id": "61f7aa22-8ff7-4399-9c3c-949e7c2c9c4f",
        }
    },
    {
        "data": {
            "source": "138",
            "target": "139",
            "id": "ac02fe17-9f8b-45eb-a6f2-d27ad936074f",
        }
    },
    {
        "data": {
            "source": "139",
            "target": "140",
            "id": "0183a155-a552-412b-933c-92b8a70faf2a",
        }
    },
    {
        "data": {
            "source": "136",
            "target": "141",
            "id": "653068f0-999d-4e5e-aed6-158a2e705899",
        }
    },
    {
        "data": {
            "source": "135",
            "target": "141",
            "id": "50d4bb69-f34f-45ca-aa97-742359847578",
        }
    },
    {
        "data": {
            "source": "141",
            "target": "142",
            "id": "827a5449-8e60-441a-9ecc-4a8afa954d98",
        }
    },
    {
        "data": {
            "source": "142",
            "target": "143",
            "id": "b2248615-bc19-4366-87bb-2b43ec4c1ab0",
        }
    },
    {
        "data": {
            "source": "143",
            "target": "144",
            "id": "8a92f788-859f-4487-b01c-8be726de5634",
        }
    },
    {
        "data": {
            "source": "140",
            "target": "145",
            "id": "5cc8115e-920b-4b01-9d17-10fa95613caa",
        }
    },
    {
        "data": {
            "source": "144",
            "target": "146",
            "id": "d7031c34-2e04-4050-8c98-260535b987ab",
        }
    },
    {
        "data": {
            "source": "146",
            "target": "147",
            "id": "52b31c8c-1995-4011-b6ef-4053258452f6",
        }
    },
    {
        "data": {
            "source": "122",
            "target": "148",
            "id": "df6892ec-67c7-4cee-ac69-13c286e2e2b0",
        }
    },
    {
        "data": {
            "source": "148",
            "target": "149",
            "id": "bf0bb98d-1a3f-454c-b503-6baad5a467b5",
        }
    },
    {
        "data": {
            "source": "149",
            "target": "150",
            "id": "1ce7af8d-ca00-4a83-bd1f-5711eebbf452",
        }
    },
    {
        "data": {
            "source": "145",
            "target": "151",
            "id": "8ed959e4-96ee-48ce-8dc6-69796e136261",
        }
    },
    {
        "data": {
            "source": "151",
            "target": "152",
            "id": "46a439ec-1218-483d-88f1-3a541fd0d794",
        }
    },
    {
        "data": {
            "source": "152",
            "target": "153",
            "id": "0134c0cc-a1fe-4c02-8e66-b31dbd8a248a",
        }
    },
    {
        "data": {
            "source": "147",
            "target": "154",
            "id": "bb91b4d8-e043-468b-98e8-8de51621bacc",
        }
    },
    {
        "data": {
            "source": "150",
            "target": "155",
            "id": "6e21b60c-2ada-43f4-b163-581b3b738c21",
        }
    },
    {
        "data": {
            "source": "155",
            "target": "156",
            "id": "67a6d48c-ead1-4a65-9361-9893e197a57e",
        }
    },
    {
        "data": {
            "source": "153",
            "target": "157",
            "id": "ff3622f1-620a-482b-8624-cdd2e9682469",
        }
    },
    {
        "data": {
            "source": "156",
            "target": "158",
            "id": "ac4522ce-3e41-455a-94cc-aa7af2a2c973",
        }
    },
    {
        "data": {
            "source": "158",
            "target": "159",
            "id": "0bd1217e-20ff-44e3-98aa-693a4864ac04",
        }
    },
    {
        "data": {
            "source": "159",
            "target": "160",
            "id": "3ae9b4cc-5485-450c-9fe6-2d1364514897",
        }
    },
    {
        "data": {
            "source": "154",
            "target": "161",
            "id": "826d5710-77d0-46c8-8cda-e0347e18ef14",
        }
    },
    {
        "data": {
            "source": "161",
            "target": "162",
            "id": "ab9ac8ba-4bbf-4616-921d-290f6f9f6a66",
        }
    },
    {
        "data": {
            "source": "162",
            "target": "163",
            "id": "02e2ac3a-9294-4919-9d50-c35a2f26d3a6",
        }
    },
    {
        "data": {
            "source": "160",
            "target": "164",
            "id": "74d82d48-8bbb-4945-bb71-5fb0f8dd7b4c",
        }
    },
    {
        "data": {
            "source": "162",
            "target": "165",
            "id": "6dcf4f5e-795e-4465-b895-c5124c7c0f27",
        }
    },
    {
        "data": {
            "source": "157",
            "target": "166",
            "id": "8d65aae6-e6bc-49c5-9478-1887d86cee43",
        }
    },
    {
        "data": {
            "source": "164",
            "target": "167",
            "id": "bd73dfe2-d5dc-46fa-8be1-06386dd7312d",
        }
    },
    {
        "data": {
            "source": "167",
            "target": "168",
            "id": "a1224745-1f97-4708-ae2d-77ec4f374a20",
        }
    },
    {
        "data": {
            "source": "167",
            "target": "169",
            "id": "ead6e539-37b2-4cff-8b84-aa81013b0da7",
        }
    },
]